In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Define path to your dataset
train_dir = '/kaggle/input/iith-dl-contest-2024/train/train'
test_dir = '/kaggle/input/iith-dl-contest-2024/test/test'


In [7]:
from PIL import Image
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

# Define mean and standard deviation for normalization
mean = [0.495, 0.455, 0.475]  # Assuming RGB images
std = [0.265, 0.255, 0.26]  

# Define transformations
train_transforms = transforms.Compose([ 
    transforms.RandomResizedCrop(64),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.GaussianBlur(3, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

val_test_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Define datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)

class_names = train_dataset.classes
num_classes = len(class_names) 

class CustomTestDataset(datasets.ImageFolder):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.samples = os.listdir(root)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root, self.samples[idx])
        img = Image.open(img_name)
        if img.mode != 'RGB':  # Convert to RGB if not already
            img = img.convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        return img, self.samples[idx]

test_dataset = CustomTestDataset(test_dir, transform=val_test_transforms)

# Split train_dataset into train and validation datasets
train_size = int(0.8 * len(train_dataset))
validation_size = len(train_dataset) - train_size
train_dataset, validation_dataset = random_split(train_dataset, [train_size, validation_size])

validation_dataset.dataset.transform = val_test_transforms
train_dataset.dataset.transform = train_transforms

# Define dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

# Define your device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define your model architecture
class MyModel(nn.Module):
    def __init__(self, num_classes):
        super(MyModel, self).__init__()
        # Load a pre-trained ResNet-152 model
        self.base_model = models.resnet152(pretrained=True)
        # Modify the last fully connected layer to match the number of classes in your dataset
        num_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return self.base_model(x)

# Define your model
model = MyModel(num_classes=num_classes)
model.to(device)

# Define your loss function
criterion = nn.CrossEntropyLoss()

# Define your optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define your learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.1)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:01<00:00, 166MB/s]  


In [9]:
# Define the number of epochs
num_epochs = 20

# Training loop
for epoch in range(num_epochs):
    # Set model to training mode
    model.train()
    
    # Initialize running loss
    running_loss = 0.0
    
    # Iterate over the training dataset
    for inputs, labels in train_loader:
        # Move inputs and labels to the appropriate device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Calculate the loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Update running loss
        running_loss += loss.item() * inputs.size(0)
    
    # Calculate average training loss for the epoch
    train_loss = running_loss / len(train_loader.dataset)
    
    # Print training loss for the epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}")
    
    # Set model to evaluation mode
    model.eval()
    
    # Initialize validation loss
    val_loss = 0.0
    
    # Initialize correct predictions and total examples
    correct_preds = 0
    total_examples = 0
    
    # Disable gradient calculation during validation
    with torch.no_grad():
        # Iterate over the validation dataset
        for inputs, labels in validation_loader:
            # Move inputs and labels to the appropriate device
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            
            # Calculate the loss
            loss = criterion(outputs, labels)
            
            # Update validation loss
            val_loss += loss.item() * inputs.size(0)
            
            # Get predicted labels
            _, predicted = torch.max(outputs, 1)
            
            # Update total examples
            total_examples += labels.size(0)
            
            # Update correct predictions
            correct_preds += (predicted == labels).sum().item()
    
    # Calculate average validation loss
    val_loss = val_loss / len(validation_loader.dataset)
    
    # Calculate validation accuracy
    val_accuracy = correct_preds / total_examples
    
    # Print validation loss and accuracy
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    
    # Adjust learning rate based on validation loss
    scheduler.step(val_loss)


Epoch 1/20, Train Loss: 3.0357
Validation Loss: 5.9551, Validation Accuracy: 0.2743
Epoch 2/20, Train Loss: 2.5979
Validation Loss: 2.3927, Validation Accuracy: 0.3596
Epoch 3/20, Train Loss: 2.3626
Validation Loss: 2.4566, Validation Accuracy: 0.3713
Epoch 4/20, Train Loss: 2.2235
Validation Loss: 2.2059, Validation Accuracy: 0.4103
Epoch 5/20, Train Loss: 2.1906
Validation Loss: 2.3653, Validation Accuracy: 0.4080
Epoch 6/20, Train Loss: 2.1306
Validation Loss: 2.1225, Validation Accuracy: 0.4372
Epoch 7/20, Train Loss: 2.0679
Validation Loss: 2.1315, Validation Accuracy: 0.4633
Epoch 8/20, Train Loss: 1.9570
Validation Loss: 2.1286, Validation Accuracy: 0.4575
Epoch 9/20, Train Loss: 1.9413
Validation Loss: 1.9905, Validation Accuracy: 0.4832
Epoch 10/20, Train Loss: 1.8081
Validation Loss: 1.8769, Validation Accuracy: 0.5029
Epoch 11/20, Train Loss: 1.8002
Validation Loss: 2.2163, Validation Accuracy: 0.4944
Epoch 12/20, Train Loss: 1.7355
Validation Loss: 1.8385, Validation Accura

In [12]:
# Define the number of epochs
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    # Set model to training mode
    model.train()
    
    # Initialize running loss
    running_loss = 0.0
    
    # Iterate over the training dataset
    for inputs, labels in train_loader:
        # Move inputs and labels to the appropriate device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Calculate the loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Update running loss
        running_loss += loss.item() * inputs.size(0)
    
    # Calculate average training loss for the epoch
    train_loss = running_loss / len(train_loader.dataset)
    
    # Print training loss for the epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}")
    
    # Set model to evaluation mode
    model.eval()
    
    # Initialize validation loss
    val_loss = 0.0
    
    # Initialize correct predictions and total examples
    correct_preds = 0
    total_examples = 0
    
    # Disable gradient calculation during validation
    with torch.no_grad():
        # Iterate over the validation dataset
        for inputs, labels in validation_loader:
            # Move inputs and labels to the appropriate device
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            
            # Calculate the loss
            loss = criterion(outputs, labels)
            
            # Update validation loss
            val_loss += loss.item() * inputs.size(0)
            
            # Get predicted labels
            _, predicted = torch.max(outputs, 1)
            
            # Update total examples
            total_examples += labels.size(0)
            
            # Update correct predictions
            correct_preds += (predicted == labels).sum().item()
    
    # Calculate average validation loss
    val_loss = val_loss / len(validation_loader.dataset)
    
    # Calculate validation accuracy
    val_accuracy = correct_preds / total_examples
    
    # Print validation loss and accuracy
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    
    # Adjust learning rate based on validation loss
    scheduler.step(val_loss)


Epoch 1/5, Train Loss: 1.5372
Validation Loss: 2.2192, Validation Accuracy: 0.4797
Epoch 2/5, Train Loss: 1.5411
Validation Loss: 1.5647, Validation Accuracy: 0.5828
Epoch 3/5, Train Loss: 1.4411
Validation Loss: 1.7368, Validation Accuracy: 0.5369
Epoch 4/5, Train Loss: 1.5146
Validation Loss: 2.0808, Validation Accuracy: 0.5346
Epoch 5/5, Train Loss: 1.4274
Validation Loss: 1.6345, Validation Accuracy: 0.5848


In [17]:
# Define the number of epochs
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    # Set model to training mode
    model.train()
    
    # Initialize running loss
    running_loss = 0.0
    
    # Iterate over the training dataset
    for inputs, labels in train_loader:
        # Move inputs and labels to the appropriate device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Calculate the loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Update running loss
        running_loss += loss.item() * inputs.size(0)
    
    # Calculate average training loss for the epoch
    train_loss = running_loss / len(train_loader.dataset)
    
    # Print training loss for the epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}")
    
    # Set model to evaluation mode
    model.eval()
    
    # Initialize validation loss
    val_loss = 0.0
    
    # Initialize correct predictions and total examples
    correct_preds = 0
    total_examples = 0
    
    # Disable gradient calculation during validation
    with torch.no_grad():
        # Iterate over the validation dataset
        for inputs, labels in validation_loader:
            # Move inputs and labels to the appropriate device
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            
            # Calculate the loss
            loss = criterion(outputs, labels)
            
            # Update validation loss
            val_loss += loss.item() * inputs.size(0)
            
            # Get predicted labels
            _, predicted = torch.max(outputs, 1)
            
            # Update total examples
            total_examples += labels.size(0)
            
            # Update correct predictions
            correct_preds += (predicted == labels).sum().item()
    
    # Calculate average validation loss
    val_loss = val_loss / len(validation_loader.dataset)
    
    # Calculate validation accuracy
    val_accuracy = correct_preds / total_examples
    
    # Print validation loss and accuracy
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    
    # Adjust learning rate based on validation loss
    scheduler.step(val_loss)


Epoch 1/5, Train Loss: 1.4054
Validation Loss: 5.1165, Validation Accuracy: 0.5050
Epoch 2/5, Train Loss: 1.5748
Validation Loss: 1.6161, Validation Accuracy: 0.5742
Epoch 3/5, Train Loss: 1.3783
Validation Loss: 1.6984, Validation Accuracy: 0.5917
Epoch 4/5, Train Loss: 1.2445
Validation Loss: 1.5253, Validation Accuracy: 0.6192
Epoch 5/5, Train Loss: 1.1967
Validation Loss: 1.7541, Validation Accuracy: 0.6207


In [18]:
# Initialize lists to store filenames and predicted categories
test_filenames = []
test_predictions = []

# Set model to evaluation mode
model.eval()

# Disable gradient calculation during testing
with torch.no_grad():
    # Iterate over the test dataset
    for inputs, filenames in test_loader:
        
        # Move inputs to the appropriate device
        inputs = inputs.to(device)
        
        # Forward pass
        outputs = model(inputs)
        
        # Get predicted labels
        _, predicted = torch.max(outputs, 1)
        
        # Convert tensor to numpy array and then to list
        filenames = [filename for filename in filenames]
        
        # Extend lists with filenames and predictions along with class names
        for filename, prediction in zip(filenames, predicted.cpu().numpy()):
            test_filenames.append(filename)
            test_predictions.append(class_names[prediction])
        
# Create a DataFrame with filenames and predicted categories
test_df = pd.DataFrame({'ID': test_filenames, 'Category': test_predictions})

# Save the DataFrame to a CSV file
test_df.to_csv('/kaggle/working/predictions.csv', index=False)
